In [1]:
import pandas as pd
import numpy as np
import json
import os
from tqdm import tqdm
from Agents.DQNAgent import DQNAgent
from Boards.Speed_leedo_2p import FullBoard
import sys
import tqdm

fetch_or_load = 1 # 0 - fetch, 1 - load

if fetch_or_load == 0:
    query="""
        select *
        from `analytics-156605.barath.ludo_details`
        order by division, time_stamp
    """

    df = pd.read_gbq(query, use_bqstorage_api=True, progress_bar_type='tqdm_notebook')

    df.to_csv('../data/ludo_2p_data_sept_10.csv')

else:
    df = pd.read_csv('../data/ludo_2p_data_sept_10.csv')

2022-09-16 10:41:50.816286: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
groups = df.sort_values(by='time_stamp').groupby('division')

In [3]:
env = FullBoard()

In [4]:
data = []
    
for n, (name, group) in enumerate(groups):
    
    group = group.reset_index()
    if n == 1:

        try:
            user_ids = []
            dice = []
            pos = []
            score = []

            skill = [group['user_skill_score'][0]]
            skill += [np.unique(group[group['from_user'] != group['from_user'][0]]['user_skill_score'])[0]]

            order = json.loads(json.loads(group['race'][0])['ludoBoardData'])['upd']
            for i, ord in enumerate(order):
                if np.sum(order[ord]['pos']) == 4 and i == 0:
                    flag = 1
                else:
                    flag = 0

            for i in  range(len(group)):

                temp_user_id = group['from_user'][i]
                temps = json.loads(json.loads(group['race'][i])['ludoBoardData'])['upd']
                temp_dice = int(group['dice_score'][i])

                for j, temp in enumerate(temps):
                    
                    if j == 0:
                        temp_pos = temps[temp]['pos']
                    else:
                        if flag:
                            temp_pos += temps[temp]['pos']
                        else:
                            temp_pos = temps[temp]['pos'] + temp_pos
                    if temp == temp_user_id:
                        if flag:
                            temp_user_id = j
                        else:
                            temp_user_id = 1 - j

                    if j == 0:
                        temp_score = [temps[temp]['totScore']]
                    else:
                        if flag:
                            temp_score += [temps[temp]['totScore']]
                        else:
                            temp_score = [temps[temp]['totScore']] + temp_score

                user_ids.append(temp_user_id)
                dice.append(temp_dice)
                pos.append(temp_pos)
                score.append(temp_score)

            user_ids = user_ids[1:]
            dice = dice[1:]

            user_ids = np.array(user_ids)
            dice = np.array(dice)
            pos = np.array(pos)
            score = np.array(score)

            action = np.argmax((pos[1:] - pos[:-1]), axis=-1) % 4

            states = np.concatenate((np.expand_dims(dice, -1), pos[:-1,:], np.zeros((len(dice),2))), -1)
            s = []
            for m in range (len(states)):
                s.append(env.convert_state(user_ids[m], env._process(states[m,:].tolist())))

            data.append({'user_ids': user_ids, 'score': score, 'states': s, 'action': action, 'skill': skill})

        except:
            continue

In [5]:
agent0 = DQNAgent(env.state_size(), env.action_size(), env.max_val())
agent0.load('./model_output/DQN_2p_v2/0004/weights_6000.hdf5')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 11, 64)            128       
_________________________________________________________________
dense_1 (Dense)              (None, 11, 64)            4160      
_________________________________________________________________
dense_2 (Dense)              (None, 11, 64)            4160      
_________________________________________________________________
dense_3 (Dense)              (None, 11, 64)            4160      
_________________________________________________________________
flatten (Flatten)            (None, 704)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 2820      
Total params: 15,428
Trainable params: 15,428
Non-trainable params: 0
____________________________________________________

2022-09-16 10:45:27.559684: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-09-16 10:45:27.560900: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-09-16 10:45:27.611094: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-16 10:45:27.612390: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: NVIDIA Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-09-16 10:45:27.612504: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
20

In [6]:
dat = data[0]
q_temp = []
q_temp_norm = []

q_vals = np.squeeze(agent0.model.predict(np.reshape(dat['states'], [len(dat['states']),11,1])/69))
for i in range (len(dat['states'])):
    q_temp.append(q_vals[i,dat['action'][i]])
    q_temp_norm.append(q_vals[i,dat['action'][i]]/np.max(q_vals[i,:]))
    

2022-09-16 10:45:32.089931: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-09-16 10:45:32.090347: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz
2022-09-16 10:45:32.240411: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


In [7]:
out = {}

out['q_user_1'] = np.array(q_temp.copy())
out['q_user_1'][dat['user_ids']==1] = 0

out['q_user_2'] = np.array(q_temp.copy())
out['q_user_2'][dat['user_ids']==0] = 0

out['q_norm_user_1'] = np.array(q_temp_norm.copy())
out['q_norm_user_1'][dat['user_ids']==1] = 0

out['q_norm_user_2'] = np.array(q_temp_norm.copy())
out['q_norm_user_2'][dat['user_ids']==0] = 0

out['Score_user_1'] = dat['score'][1:,0]
out['Score_user_2'] = dat['score'][1:,1]

out['Dice Role'] = np.array(dat['states'])[:,0]

In [8]:
len(out['Score_user_1'])

105

In [9]:
out = pd.DataFrame(out)

In [10]:
out.to_csv('../data/one_game_v2.csv')